In [ ]:
import instructor
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
from dotenv import load_dotenv

load_dotenv()

# Enables `response_model`
client = instructor.patch(OpenAI())

In [ ]:


class UserDetail(BaseModel):
    name: str
    age: int

user = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=UserDetail,
    messages=[
        {"role": "user", "content": "Extract Brian is 46 years old"},
    ]
)

assert isinstance(user, UserDetail)
assert user.name == "Brian"
assert user.age == 46

json_str = user.model_dump_json()
print(json_str)

In [ ]:

class Website(BaseModel):
    name: str
    url: str

class WebsiteExtraction(BaseModel):
    websites: List[Website] = Field(
        ...,
        description="Body of the answer, each website should be a separate object with a name and a url of the website",
    )

def get_websites(prompt) -> WebsiteExtraction:
    website_extraction: WebsiteExtraction = client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_model=WebsiteExtraction,
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    return '\n'.join([f"{website.name}, {website.url}" for website in website_extraction.websites])
    

#print(get_websites("Extract the top 20 technology websites"), "\n")
print(get_websites("""
    Extract the top 20 AWS blog websitess.
    The website should not be owned by AWS the company.
    The website URL can not have 'amazon.com' as part of the website url
"""), "\n")